In [4]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
plt.style.use('ggplot')
%pylab inline
import seaborn as sns

from tic_tac_toe import *

Populating the interactive namespace from numpy and matplotlib


In [63]:
class ClassicBoard():
    
    def __init__(self, board_id):
        self.board = np.array(np.zeros((3,3)))
        self.board_id = board_id
        
        self.won = False
        self.winner = ''
        self.game_over = False
        
        self.remaining_squares = set(range(9))
        
        self._lab2int = {'X':1, 'O':2}
        self._int2lab = {1:'X', 2:'O', 0:'-'}
        
        self.history = []
        
    def play(self, pos, label):
        if self.game_over:
            raise ValueError(f"The game on board {self.board_id} has ended")
            
        # convert integer plays to tuples
        if isinstance(pos, (int, np.int64)):
            pos_tup = (math.floor(pos / 3), pos % 3)
            pos_int = pos
        # convert tuple plays to integers 
        if isinstance(pos, tuple):
            pos_tup = pos
            pos_int = 3*pos[0] + pos[1]
        
        # Raise errors for previously played squares
        if self.board[pos_tup[0], pos_tup[1]] > 0:
            raise ValueError(f"Position {(pos_tup[0],pos_tup[1])} on board {self.board_id} has already been played")
        
        # Check for correct labels
        if label not in {'X','O'}:
            raise ValueError("Label must be in {'X','O'}")
        
        # make play, add to history
        self.board[pos_tup[0], pos_tup[1]] = self._lab2int[label]
        self.history.append((pos_tup, self._lab2int[label]))
        
        self._update_state(pos_int, label)
        
    def _update_state(self, pos, label):
        '''
        pos: 
        '''
        self.remaining_squares.remove(pos)
        
        
        # generate boolean board where squares == 1 if it matches
        # the last play and 0 otherwise
        label_board = self.board == self._lab2int[label]
        
        if (np.any(np.sum(label_board, axis=0) == 3) or
            np.any(np.sum(label_board, axis=1) == 3) or
            sum([label_board[i,i] for i in range(3)]) == 3 or
            sum([label_board[i,2-i] for i in range(3)]) == 3):
            # if a player has won, set appropriate flags
            self.won = True
            self.winner = label
            self.game_over = True
            
        if len(self.remaining_squares) == 0:
            self.game_over = True
            
    def undo_move(self):
        if len(self.history) == 0:
            return
        
        last_move, last_player = self.history[-1]
        self.history = self.history[:-1]
        self.game_over = False
        self.board[(last_move[0], last_move[1])] = 0
        self.remaining_squares.add(last_move[0]*3 + last_move[1])
        
    def is_winnable(self):
        if self.won: return False
        
        triplets = [[0,1,2], [3,4,5], [6,7,8],
                    [0,3,6], [1,4,7], [2,5,8],
                    [0,4,8], [2,4,6]]
        
        flat_board = self.board.flatten()
        
        for t in triplets:
            n_unq = len(np.unique(flat_board[t]))
            # all empty squares
            if n_unq == 1:
                return True
            # an X, an O and an empty - can't win this triple
            elif n_unq == 3:
                continue
            # Two unique values has 3 cases:
            # 1. Xs and Os - not winnable
            # 2. Blanks and Xs - winnable
            # 3. Blanks and Os - winnable
            # Rule is therefore if any blank, then winnable
            else:
                if np.any(flat_board[t]==0):
                    return True
                else:
                    continue
                
        return False
                
    
    def __str__(self):
        lines = []
        for i in range(3):
            lines.append(' '.join([self._int2lab[num] for num in self.board[i,:]]))
            
        return '\n'.join(lines)

In [64]:
board = ClassicBoard(0)

In [68]:
board = ClassicBoard(0)
board.play(0, 'O')
board.play(2, 'X')
board.play(3, 'X')
board.play(4, 'O')
board.play(5, 'O')
board.play(6, 'O')
board.play(8, 'X')

In [71]:
board.play(7,'X')

In [72]:
board.is_winnable()

False

In [73]:
print(board)

O - X
X O O
O X X


In [53]:
print(board.is_winnable())

[ 1.  0.  0.] 2
[ 0.  0.  0.] 1
[ 0.  0.  0.] 1
[ 1.  0.  0.] 2
[ 0.  0.  0.] 1
[ 0.  0.  0.] 1
[ 1.  0.  0.] 2
[ 0.  0.  0.] 1
None


In [9]:
board.play((1,1),'O')

In [10]:
print(board)

X - -
- O -
- - -


In [11]:
board.play((2,2), 'X')

In [12]:
print(board)

X - -
- O -
- - X


In [13]:
board.history

[((0, 0), 1), ((1, 1), 2), ((2, 2), 1)]

In [14]:
board.undo_move()

In [83]:
board.history

[((0, 0), 1), ((1, 1), 2)]

In [15]:
print(board)

X - -
- O -
- - -


In [16]:
board.play((2,0),'X')

In [17]:
print(board)

X - -
- O -
X - -


In [18]:
board.play(3, 'X')

In [19]:
print(board)

X - -
X O -
X - -


In [20]:
board.game_over

True

In [21]:
board.undo_move()

In [22]:
board.play((0,2),'X')

In [23]:
print(board)

X - X
- O -
X - -


In [24]:
board.play((0,1),'O')

In [25]:
print(board)

X O X
- O -
X - -


In [26]:
board.play(,'X')

In [27]:
print(board)

X O X
- O X
X - -


In [28]:
board.play(8, 'X')

In [29]:
print(board)

X O X
- O X
X - X


In [30]:
board.play(7,'O')

ValueError: The game on board 0 has ended